Librerias Necesarias

In [15]:
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import cv2

Funciones a Utilizar

In [16]:
def cargar_imagenes(carpeta, label):
    datos_imagenes = []
    archivos = sorted(os.listdir(carpeta))
    for nombre_archivo in archivos:
        if nombre_archivo.endswith(".png"):
            path = os.path.join(carpeta, nombre_archivo)
            imagen = plt.imread(path)  # Cargar la imagen como array
            datos_imagenes.append((label, imagen))
    return datos_imagenes

In [17]:
# Preprocesamiento como espera ResNet (ImagenNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Tamaño estándar para ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Valores de ImageNet
                        std=[0.229, 0.224, 0.225])
])

# Modelo ResNet18 sin capa final de clasificación
resnet = models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Quitar capa FC final
resnet.eval()

def generar_embeddings(carpeta_entrada, label='Default'):
    if not os.path.exists(carpeta_entrada):
        raise FileNotFoundError(f'La carpeta {carpeta_entrada} no existe.')
    
    datos_imagenes = cargar_imagenes(carpeta_entrada, label)
    embeddings = []
    for nombre, imagen in datos_imagenes:
        # Asegurar formato correcto
        if imagen.dtype != np.uint8:
            imagen = (imagen * 255).clip(0, 255).astype(np.uint8)

        if len(imagen.shape) == 2:
            # Escala de grises → convertir a RGB
            imagen = cv2.cvtColor(imagen, cv2.COLOR_GRAY2RGB)
        elif imagen.shape[-1] == 4:
            # RGBA → eliminar canal alfa
            imagen = imagen[:, :, :3]

        # Convertir a PIL.Image
        imagen_pil = Image.fromarray(imagen)

        # Transformar y generar embedding
        imagen_tensor = transform(imagen_pil).unsqueeze(0)
        with torch.no_grad():
            emb = resnet(imagen_tensor).squeeze().numpy()

        embeddings.append((label, emb))
    
    return embeddings



c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
def obtener_val_vector(imagen): # Recibe imagen BGR o Gray
    # --- Preparar imagen ---
    if len(imagen.shape) == 3 and imagen.shape[2] == 3:
        img_gray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
        img_hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    else:
        img_gray = imagen
        img_hsv = None

    total_pixeles = img_gray.size

    # 1. Media de intensidad
    media = np.mean(img_gray)

    # 2. Desviación estándar de intensidad
    std = np.std(img_gray)

    # 3–6. Histograma reducido (4 bins)
    hist, _ = np.histogram(img_gray, bins=4, range=(0, 256), density=True)
    hist = hist.tolist()

    # 7. Porcentaje de pixeles oscuros (< 50)
    porc_oscuro = np.sum(img_gray < 50) / total_pixeles

    # 8. Porcentaje de pixeles claros (> 200)
    porc_claro = np.sum(img_gray > 200) / total_pixeles

    # 9. Densidad de bordes (Canny)
    img_gray = img_gray.astype(np.uint8)
    bordes = cv2.Canny(img_gray, 100, 200)
    dens_bordes = np.sum(bordes > 0) / total_pixeles

    # 10. Número de componentes conectados (Otsu)
    # --- Convertir imagen a escala de grises si tiene más de 1 canal ---
    if len(imagen.shape) == 3:
        if imagen.shape[2] == 4:  # RGBA
            imagen = imagen[:, :, :3]  # eliminar canal alfa
        img_gray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    else:
        img_gray = imagen
    # Asegurar tipo uint8
    if img_gray.dtype != np.uint8:
        img_gray = (img_gray * 255).clip(0, 255).astype(np.uint8)
    _, binarizada = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    n_labels, labels = cv2.connectedComponents(binarizada)
    num_componentes = n_labels - 1  # sin contar el fondo

    # 11. Área del componente más grande (excluyendo fondo)
    if num_componentes > 0:
        stats = cv2.connectedComponentsWithStats(binarizada)[2]
        area_max = np.max(stats[1:, cv2.CC_STAT_AREA])
    else:
        area_max = 0

    # 12–13. Centroide de masa (normalizado)
    masa_total = np.sum(img_gray)
    if masa_total > 0:
        y_indices, x_indices = np.indices(img_gray.shape)
        cx = np.sum(x_indices * img_gray) / masa_total
        cy = np.sum(y_indices * img_gray) / masa_total
        cx /= img_gray.shape[1]
        cy /= img_gray.shape[0]
    else:
        cx = cy = 0.5

    # 14. Razón de aspecto
    razon_aspecto = img_gray.shape[1] / img_gray.shape[0]

    # 15. Color dominante (modo del canal H de HSV)
    if img_hsv is not None:
        hist_h = cv2.calcHist([img_hsv], [0], None, [180], [0, 180])
        color_dominante = np.argmax(hist_h) / 179  # normalizado
    else:
        color_dominante = 0.0

    # Armar el vector
    vector = [
        media,
        std,
        *hist,  # 4 bins
        porc_oscuro,
        porc_claro,
        dens_bordes,
        num_componentes,
        area_max,
        cx,
        cy,
        razon_aspecto,
        color_dominante
    ]

    return vector


In [19]:
def generar_vector_caracteristico(carpeta_entrada, label='Default'):
    if not os.path.exists(carpeta_entrada):
        raise FileNotFoundError(f'La carpeta {carpeta_entrada} no existe.')
    
    datos_imagenes = cargar_imagenes(carpeta_entrada, label)
    vec = []
    for nombre, imagen in datos_imagenes:
        vector = obtener_val_vector(imagen)
        vec.append((label, vector))
    
    return vec

Obtención de los vectores y embeddings

In [20]:
embeddings_esp = []
embeddings_esp.extend(generar_embeddings('Espectrogramas_Audios_Buenos', 'Buena_pronunciacion'))
embeddings_esp.extend(generar_embeddings('Espectrogramas_Audios_Malos', 'Mala_pronunciacion'))

embeddings_label = [nombre for nombre, _ in embeddings_esp]
solo_vectores = [vec for _, vec in embeddings_esp]
embeddings_esp = np.array(solo_vectores)
embeddings_label = np.array(embeddings_label)

print("Embeddings, shapes:")
print(embeddings_esp.shape)
print("Labels:")
print(embeddings_label.shape)

Embeddings, shapes:
(2173, 512)
Labels:
(2173,)


In [21]:
vector_esp = []
vector_esp.extend(generar_vector_caracteristico('Espectrogramas_Audios_Buenos', 'Buena_pronunciacion'))
vector_esp.extend(generar_vector_caracteristico('Espectrogramas_Audios_Malos', 'Mala_pronunciacion'))

vector_label = [nombre for nombre, _ in vector_esp]
solo_vectores = [vec for _, vec in vector_esp]
vector_esp = np.array(solo_vectores)
vector_label = np.array(vector_label)

print("Vector Caracteristico, shapes:")
print(vector_esp.shape)
print("Labels:")
print(vector_label.shape)

Vector Caracteristico, shapes:
(2173, 15)
Labels:
(2173,)
